# Task7 缺失数据

## 1 知识梳理（重点记忆）

### 1.1 缺失值的统计与删除
- 缺失信息的统计：主要使用`isna().sum()`查看缺失的比例
- 删除缺失信息：使用`dropna()`函数，其中`thresh`参数表示非缺失值 没有达到这个数量的相应维度会被删除

### 1.2 缺失值的填充和插值
- `fillna`函数：`limit`参数表示连续缺失值的最大填充次数
- `interpolate`函数：`limit_direction`参数表示控制方向，`limit`参数表示连续缺失值的最大填充次数

### 1.3 Nullable类型
- `None`除了等于自己本身之外，与其他任何元素不相等
- `np.nan`与其他任何元素不相等
- 在使用`equals`函数比较两张表时，会自动跳过两张表都是缺失值的位置
- 在时间序列对象中，使用`pd.NaT`表示缺失值

### 1.4 缺失数据的计算和分组
- 进行`sum`和`prod`时，缺失数据不影响计算
- 当使用累计函数（例如`cumsum`）时，会自动跳过缺失值所处的位置
- 在使用`groupby`、`get_dumies`函数时，通过设置`dropna=False`，缺失值可以作为一个类别

## 2 练一练

### 2.1 第1题

对一个序列以如下规则填充缺失值：如果单独出现的缺失值，就用前后均值填充，如果连续出现的缺失值就不填充，即序列`[1, NaN, 3, NaN, NaN]`填充后为`[1, 2, 3, NaN, NaN]`，请利用`fillna`函数实现。（提示：利用`limit`参数）

**我的解答：**

In [1]:
import pandas as pd
import numpy as np

In [2]:
s = pd.Series([np.nan, 1, np.nan, 3, 4, np.nan, 4, np.nan, np.nan, 5])
s

0    NaN
1    1.0
2    NaN
3    3.0
4    4.0
5    NaN
6    4.0
7    NaN
8    NaN
9    5.0
dtype: float64

In [3]:
# 分别使用ffill和bfill，并limit限制为1，得到两个Series
s_ffill = s.fillna(method='ffill', limit=1)
s_bfill = s.fillna(method='bfill', limit=1)

In [4]:
# 构造一个DataFrame
pd.DataFrame([s_ffill, s_bfill])

,0,1,2,3,4,5,6,7,8,9
0,NaN,1.0,1.0,3.0,4.0,4.0,4.0,4.0,NaN,5.0
1,1.0,1.0,3.0,3.0,4.0,4.0,4.0,NaN,5.0,5.0


In [5]:
# 对DataFrame求mean，其中skipna表示求mean的时候不忽略nan
pd.DataFrame([s_ffill, s_bfill]).mean(axis=0, skipna=False)

0    NaN
1    1.0
2    2.0
3    3.0
4    4.0
5    4.0
6    4.0
7    NaN
8    NaN
9    5.0
dtype: float64

## 3 练习

### 3.1 Ex1：缺失值与类别的相关性检验
在数据处理中，含有过多缺失值的列往往会被删除，除非缺失情况与标签强相关。下面有一份关于二分类问题的数据集，其中`X_1, X_2`为特征变量，`y`为二分类标签。

In [6]:
df = pd.read_csv('../data/missing_chi.csv')
df.head()

,X_1,X_2,y
0,NaN,NaN,0
1,NaN,NaN,0
2,NaN,NaN,0
3,43.0,NaN,0
4,NaN,NaN,0


In [7]:
df.isna().mean()

X_1    0.855
X_2    0.894
y      0.000
dtype: float64

In [8]:
df.y.value_counts(normalize=True)

0    0.918
1    0.082
Name: y, dtype: float64

事实上，有时缺失值出现或者不出现本身就是一种特征，并且在一些场合下可能与标签的正负是相关的。关于缺失出现与否和标签的正负性，在统计学中可以利用卡方检验来断言它们是否存在相关性。按照特征缺失的正例、特征缺失的负例、特征不缺失的正例、特征不缺失的负例，可以分为四种情况，设它们分别对应的样例数为$n_{11}, n_{10}, n_{01}, n_{00}$。假若它们是不相关的，那么特征缺失中正例的理论值，就应该接近于特征缺失总数$\times$总体正例的比例，即：

$$E_{11} = n_{11} \approx (n_{11}+n_{10})\times\frac{n_{11}+n_{01}}{n_{11}+n_{10}+n_{01}+n_{00}} = F_{11}$$

其他的三种情况同理。现将实际值和理论值分别记作$E_{ij}, F_{ij}$，那么希望下面的统计量越小越好，即代表实际值接近不相关情况的理论值：

$$S = \sum_{i\in \{0,1\}}\sum_{j\in \{0,1\}} \frac{(E_{ij}-F_{ij})^2}{F_{ij}}$$

可以证明上面的统计量近似服从自由度为$1$的卡方分布，即$S\overset{\cdot}{\sim} \chi^2(1)$。因此，可通过计算$P(\chi^2(1)>S)$的概率来进行相关性的判别，一般认为当此概率小于$0.05$时缺失情况与标签正负存在相关关系，即不相关条件下的理论值与实际值相差较大。

上面所说的概率即为统计学上关于$2\times2$列联表检验问题的$p$值， 它可以通过`scipy.stats.chi2(S, 1)`得到。请根据上面的材料，分别对`X_1, X_2`列进行检验。

**我的解答：**  

通过该题，可利用`pd.crosstab`方法统计元素组合[(NaN, 1), (NaN, 0), (NotNaN, 1), (NotNaN, 0)]出现的频数

In [9]:
# 把所有为NaN的值标记为NaNValue，其他值标记为NotNaNValue
df_X1_flag = df['X_1'].fillna("NaNValue").mask(df['X_1'].notna()).fillna("NotNaNValue")
df_1 = pd.crosstab(df_X1_flag, df['y'], margins=True)
df_1

y,0,1,All
X_1,,,
NaNValue,785,70,855
NotNaNValue,133,12,145
All,918,82,1000


In [10]:
df_X2_flag = df['X_2'].fillna("NaNValue").mask(df['X_2'].notna()).fillna("NotNaNValue")
df_2 = pd.crosstab(df_X2_flag, df['y'], margins=True)
df_2

y,0,1,All
X_2,,,
NaNValue,894,0,894
NotNaNValue,24,82,106
All,918,82,1000


In [11]:
def compute_S(df):
    res = []
    for i in [0, 1]:
        for j in [0, 1]:
            E = df.iloc[i, j]
            F = df.iloc[i, 2] * df.iloc[2, j] / df.iloc[2, 2]
            res.append((E-F)**2/F)
    return sum(res)

In [12]:
from scipy.stats import chi2

chi2.sf(compute_S(df_1), 1)

0.9712760884395901

In [13]:
chi2.sf(compute_S(df_2), 1)

7.459641265637543e-166

可知X1的概率大于0.05，X2的概率小于0.05，故特征X2的缺失情况与标签正负存在相关关系，不能删除，特征X1可以删除

### 3.2 Ex2：用回归模型解决分类问题

`KNN`是一种监督式学习模型，既可以解决回归问题，又可以解决分类问题。对于分类变量，利用`KNN`分类模型可以实现其缺失值的插补，思路是度量缺失样本的特征与所有其他样本特征的距离，当给定了模型参数`n_neighbors=n`时，计算离该样本距离最近的$n$个样本点中最多的那个类别，并把这个类别作为该样本的缺失预测类别，具体如下图所示，未知的类别被预测为黄色：

<img src="../source/_static/ch7_ex.png" width="25%">

上面有色点的特征数据提供如下：

In [14]:
df = pd.read_excel('../data/color.xlsx')
df.head(3)

,X1,X2,Color
0,-2.5,2.8,Blue
1,-1.5,1.8,Blue
2,-0.8,2.8,Blue


已知待预测的样本点为$X_1=0.8, X_2=-0.2$，那么预测类别可以如下写出：

In [15]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=6)
clf.fit(df.iloc[:,:2], df.Color)
clf.predict([[0.8, -0.2]])

array(['Yellow'], dtype=object)

1. 对于回归问题而言，需要得到的是一个具体的数值，因此预测值由最近的$n$个样本对应的平均值获得。请把上面的这个分类问题转化为回归问题，仅使用`KNeighborsRegressor`来完成上述的`KNeighborsClassifier`功能。
2. 请根据第1问中的方法，对`audit`数据集中的`Employment`变量进行缺失值插补。

In [16]:
df = pd.read_csv('../data/audit.csv')
df.head(3)

,ID,Age,Employment,Marital,Income,Gender,Hours
0,1004641,38,Private,Unmarried,81838.00,Female,72
1,1010229,35,Private,Absent,72099.00,Male,30
2,1024587,32,Private,Divorced,154676.74,Male,40


**我的解答：**

**第1问：**

In [17]:
from sklearn.neighbors import KNeighborsRegressor

In [18]:
df = pd.read_excel('../data/color.xlsx')

In [19]:
# 进行one-hot编码
df_color = pd.get_dummies(df['Color'])

# 得到所有类别的概率
res = []
for color in df_color.columns:
    clf = KNeighborsRegressor(n_neighbors=6)
    clf.fit(df.iloc[:,:2], df_color[color])
    predict_value = clf.predict([[0.8, -0.2]])[0]
    res.append(predict_value)

In [20]:
res

[0.16666666666666666, 0.3333333333333333, 0.5]

In [21]:
# 取出概率最大的那个类别，即为所求
df_color.columns[np.array(res).argmax()]

'Yellow'

**第2问：**

In [22]:
df = pd.read_csv('../data/audit.csv')
df.head(3)

,ID,Age,Employment,Marital,Income,Gender,Hours
0,1004641,38,Private,Unmarried,81838.00,Female,72
1,1010229,35,Private,Absent,72099.00,Male,30
2,1024587,32,Private,Divorced,154676.74,Male,40


In [23]:
my_df = df.copy()

In [24]:
# 对Age，Income，Hours进行归一化处理
df_normalize = my_df[['Age','Income','Hours']].apply(lambda x:(x-x.min())/(x.max()-x.min()))

In [25]:
# 对Marital，Gender进行one-hot编码
df_one_hot = pd.get_dummies(my_df[['Marital', 'Gender']])

In [26]:
# 构造类似于第1问中的数据集
my_df = pd.concat([df_normalize, df_one_hot, df['Employment']], axis=1)

In [27]:
my_df.head()

,Age,Income,Hours,Marital_Absent,Marital_Divorced,Marital_Married,Marital_Married-spouse-absent,Marital_Unmarried,Marital_Widowed,Gender_Female,Gender_Male,Employment
0,0.287671,0.168997,0.724490,0,0,0,0,1,0,1,0,Private
1,0.246575,0.148735,0.295918,1,0,0,0,0,0,0,1,Private
2,0.205479,0.320539,0.397959,0,1,0,0,0,0,0,1,Private
3,0.383562,0.056453,0.551020,0,0,1,0,0,0,0,1,Private
4,0.589041,0.014477,0.397959,0,0,1,0,0,0,0,1,Private


In [28]:
X_train = my_df[my_df.Employment.notna()]
X_test = my_df[my_df.Employment.isna()]

In [29]:
# 进行one-hot编码
df_employment = pd.get_dummies(X_train['Employment'])

# 得到所有类别的概率
res = []
for employment in df_employment.columns:
    clf = KNeighborsRegressor(n_neighbors=6)
    clf.fit(X_train.iloc[:,:-1], df_employment[employment])
    predict_value = clf.predict(X_test.iloc[:,:-1]).reshape(-1,1)
    res.append(predict_value)

In [30]:
# 得到该行的最大值索引
np.hstack(res).argmax(1)

array([2, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 0, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4], dtype=int64)

In [31]:
# 构造成为一个pd.Series
Y_test = pd.Series(df_employment.columns[pd.Series(np.hstack(res).argmax(1))].values)

In [32]:
# 将Employment为nan的值进行赋值
df.loc[df.Employment.isna(), 'Employment'] = Y_test.values

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          2000 non-null   int64  
 1   Age         2000 non-null   int64  
 2   Employment  2000 non-null   object 
 3   Marital     2000 non-null   object 
 4   Income      2000 non-null   float64
 5   Gender      2000 non-null   object 
 6   Hours       2000 non-null   int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 109.5+ KB
